In [ ]:
pip install download

In [ ]:
# importing all libraries
from download import download
import pandas as pd
import numpy as np

In [ ]:
# downloading the data from the location and storing it in the path tmp/aq
path = download('https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip','/tmp/aq', kind = 'zip')

Creating data folder...

file_sizes: 100%|██████████████████████████| 16.8M/16.8M [00:00<00:00, 86.2MB/s]
Extracting zip file...
Successfully downloaded / unzipped to /tmp/aq


In [ ]:
# storing the data as mentioned in the question
data = pd.read_csv('/tmp/aq/201306-citibike-tripdata.csv') \
                  [['tripduration', 'starttime', 'start station name', \
                                 'end station name', 'bikeid', 'usertype']] \
.rename(columns = {'start station name':'start_station_name', 'end station name':'end_station_name'})
data = data[data.end_station_name.notnull()] #removing not null values from end_station_name

In [ ]:
print(data.shape)
data.head()

(559644, 6)


,tripduration,starttime,start_station_name,end_station_name,bikeid,usertype
0,695,2013-06-01 00:00:01,Broadway & W 24 St,9 Ave & W 18 St,19678,Subscriber
1,693,2013-06-01 00:00:08,Broadway & W 24 St,9 Ave & W 18 St,16649,Subscriber
2,2059,2013-06-01 00:00:44,Hicks St & Montague St,Hicks St & Montague St,19599,Customer
3,123,2013-06-01 00:01:04,E 15 St & Irving Pl,Washington Park,16352,Subscriber
4,1521,2013-06-01 00:01:22,Little West St & 1 Pl,State St & Smith St,15567,Subscriber


#### **Question 1**

Create a dataframe that - for each bike - tells us how many trips it was used for and the average duration of those trips.

In [ ]:
trips = pd.DataFrame() #creating a dataframe
trips['no_of_trips'] = data.groupby("bikeid")["bikeid"].count() #finding the number of trips by each bike
trips['avg_duration'] = data.groupby("bikeid")["tripduration"].mean() #avg duration of the trips

In [ ]:
# testing for bikeid 14529
print(data[data['bikeid']==14529].shape)
print(data[data['bikeid']==14529]['tripduration'].sum(axis=0)/data[data['bikeid']==14529].shape[0])

(51, 6)
2096.9803921568628


In [ ]:
trips.head()

,no_of_trips,avg_duration
bikeid,,
14529,51,2096.980392
14531,142,1112.408451
14532,146,875.376712
14533,87,1276.574713
14534,121,1624.355372


#### **Question 2**

The usertype column describes the user type for that trip. For "Subscribers", trips shorter than 45 minutes are free, and trips over 45 minutes incur a fee. For "Customers", trips shorter than 30 minutes are free, and trips over 30 minutes incur a fee. Create a dataframe that - for each station - lists the proportion of trips started at that station that were free.

In [ ]:
#function for creating an indicator for the free rides
def f(row):
  if (row['tripduration'] < 2700) & (row['usertype'] == 'Subscriber'):
    val = "Free"
  elif (row['tripduration'] < 1800) & (row['usertype'] == 'Customer'):
    val = "Free"
  else:
    val = "Not Free"
  return val

In [ ]:
data['free_trip_ind'] = data.apply(f, axis=1) #applying the function to the dataframe

In [ ]:
station_name = data.groupby(["start_station_name","free_trip_ind"]).agg({'bikeid': 'count'}) #grouping the data and finding the count based on the starting station and the free rides
trip_pct = station_name.groupby(level=0).apply(lambda x:100 * x / float(x.sum())) #finding the percentage for both free and not free rides
trip_pct.rename(columns = {'bikeid':'proportion'}, inplace = True) 
trip_pct = trip_pct.reset_index()

In [ ]:
free_trip_df = trip_pct[trip_pct['free_trip_ind'] == 'Free'] #selecting the free rides
free_trip_df = free_trip_df[['start_station_name','proportion']]

In [ ]:
free_trip_df.head()

,start_station_name,proportion
0,1 Ave & E 16 St,95.640569
2,1 Ave & E 18 St,94.128440
4,1 Ave & E 30 St,93.571862
6,1 Ave & E 44 St,91.082045
8,10 Ave & W 28 St,92.548435


#### **Question 3**

For each station, we define the station's "drop off popularity" as the number of times a bike was dropped off at that station in our data. For each station, we define the station's "proxy popularity" by looking at all the trips *originating* from that station, and averaging the drop off popularity of the stations these trips were ended at. For example, suppose station X had three trips in our data; one trip ending at station A (drop off popularity 1500), one ending at station B (drop off popularity 1200), and one ending at station C (drop off popularity 1600), then the proxy popularity for station X is (1500 + 1200 + 1600)/3 = 1433.33. Create a dataframe that - for each station - finds its proxy popularity.

In [ ]:
#finding the drop off popularity by counting the number of bikes that terminate at a particular station
drop_off_df = pd.DataFrame()
drop_off_df['drop_off_popularity'] = data.groupby("end_station_name")["bikeid"].count()

In [ ]:
#merging the above dataframe with the calculated drop_off_popularity with the orignal one
df_final = pd.merge(data, drop_off_df, on='end_station_name')

In [ ]:
#finding the mean of the the drop_off_popularity for each starting station
proxy_pop_df = df_final.groupby("start_station_name")["drop_off_popularity"].mean()

In [ ]:
proxy_pop_df = proxy_pop_df.reset_index()

In [ ]:
proxy_pop_df.head()

,start_station_name,drop_off_popularity
0,1 Ave & E 16 St,2327.218861
1,1 Ave & E 18 St,2331.233028
2,1 Ave & E 30 St,2227.709521
3,1 Ave & E 44 St,2271.000595
4,10 Ave & W 28 St,2469.861401


#### **Question 4**


Fit a linear regression model to predict the number of bikes that will be picked up from a station in a given hour based on the hour of day, day of week, and the number of bikes that were picked up from that station in the previous hour. Fit the model on the whole dataset - do not use separate training/test sets.

A colleague of yours suggests that every night, you should use the linear regression model you just created to estimate the number of bikes that will be required at each station the next day, and use it to re-balance bikes across the system. What is the glaring flaw in this suggestion?

In [ ]:
data.head()

,tripduration,starttime,start_station_name,end_station_name,bikeid,usertype,free_trip_ind
0,695,2013-06-01 00:00:01,Broadway & W 24 St,9 Ave & W 18 St,19678,Subscriber,Free
1,693,2013-06-01 00:00:08,Broadway & W 24 St,9 Ave & W 18 St,16649,Subscriber,Free
2,2059,2013-06-01 00:00:44,Hicks St & Montague St,Hicks St & Montague St,19599,Customer,Not Free
3,123,2013-06-01 00:01:04,E 15 St & Irving Pl,Washington Park,16352,Subscriber,Free
4,1521,2013-06-01 00:01:22,Little West St & 1 Pl,State St & Smith St,15567,Subscriber,Free


In [ ]:
data['date'] = pd.to_datetime(data['starttime']).dt.date #extracting the date 
data['day_of_week'] = pd.to_datetime(data['starttime']).dt.day_name() #extracting the day of the week
data['datehour'] = pd.to_datetime(data['starttime']).dt.hour #extracting hour

In [ ]:
#the number of bikes taken from each station on a particular date from a given start station 
regress_df = data.groupby(["start_station_name","date","day_of_week","datehour"])["bikeid"].count()
regress_df = regress_df.reset_index()

In [ ]:
regress_df.head()

,start_station_name,date,day_of_week,datehour,bikeid
0,1 Ave & E 16 St,2013-06-01,Saturday,0,1
1,1 Ave & E 16 St,2013-06-01,Saturday,9,3
2,1 Ave & E 16 St,2013-06-01,Saturday,10,1
3,1 Ave & E 16 St,2013-06-01,Saturday,12,3
4,1 Ave & E 16 St,2013-06-01,Saturday,14,1


In [ ]:
# calculating the time diff with the previous row to evaluate only those rows where the time diff is 1
regress_df['timediff'] = regress_df.sort_values(by=['datehour'],ascending=False).groupby(["start_station_name","date"])\
["datehour"].diff(-1)

In [ ]:
#testing for the above case
# regress_df[(pd.to_datetime(regress_df['date']) == '2013-06-01') & (regress_df['start_station_name'] == '1 Ave & E 16 St')] 

In [ ]:
lag_df = regress_df[regress_df['timediff'] == 1]
no_lag_df = regress_df[regress_df['timediff'] != 1]

In [ ]:
#creating a column which holds the value of the previous hour no of bikes
lag_df['lagged'] = (lag_df.sort_values(by=['datehour'], ascending=True)
                        .groupby(['start_station_name','date'])['bikeid'].shift(1))
no_lag_df['lagged'] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
lag_df.head()

,start_station_name,date,day_of_week,datehour,bikeid,timediff,lagged
2,1 Ave & E 16 St,2013-06-01,Saturday,10,1,1.0,NaN
5,1 Ave & E 16 St,2013-06-01,Saturday,15,1,1.0,1.0
6,1 Ave & E 16 St,2013-06-01,Saturday,16,1,1.0,1.0
8,1 Ave & E 16 St,2013-06-01,Saturday,19,2,1.0,1.0
9,1 Ave & E 16 St,2013-06-01,Saturday,20,4,1.0,2.0


In [ ]:
final_df = pd.concat([lag_df, no_lag_df])
final_df["lagged"].fillna(0, inplace=True)
final_df = final_df[['start_station_name','day_of_week','datehour','lagged','bikeid']].sort_values\
(['start_station_name','day_of_week','datehour'])

In [ ]:
df = pd.DataFrame()
df['lag_no_of_bikes'] = final_df.groupby(['start_station_name','day_of_week','datehour'])["lagged"].mean()
df["no_of_bikes"] = final_df.groupby(['start_station_name','day_of_week','datehour'])["bikeid"].mean()
df.reset_index(inplace=True)

In [ ]:
df.head(10)

,start_station_name,day_of_week,datehour,lag_no_of_bikes,no_of_bikes
0,1 Ave & E 16 St,Friday,0,0.000000,3.000000
1,1 Ave & E 16 St,Friday,2,0.000000,1.500000
2,1 Ave & E 16 St,Friday,4,0.000000,1.000000
3,1 Ave & E 16 St,Friday,6,0.000000,2.500000
4,1 Ave & E 16 St,Friday,7,0.000000,5.000000
5,1 Ave & E 16 St,Friday,8,4.666667,15.666667
6,1 Ave & E 16 St,Friday,9,15.666667,15.333333
7,1 Ave & E 16 St,Friday,10,15.333333,1.666667
8,1 Ave & E 16 St,Friday,11,1.666667,2.000000
9,1 Ave & E 16 St,Friday,12,1.500000,3.000000


In [ ]:
#creating the station name merged with the day of the week and datehour as the index
df["inde_x"] = df["start_station_name"] + " | " + df["day_of_week"] + " | " + df["datehour"].astype(str)
df = df.set_index('inde_x')
df.drop(['start_station_name'],axis = 1,inplace = True)

**Label Encoding for Weeks**

In [ ]:
#converting the weekdays into labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['day_of_week_cat'] = labelencoder.fit_transform(df['day_of_week'])

In [ ]:
#storing the values for future reference
weekday_dict = pd.Series(df.day_of_week.values,index=df.day_of_week_cat).to_dict()
weekday_dict

{0: 'Friday',
 1: 'Monday',
 2: 'Saturday',
 3: 'Sunday',
 4: 'Thursday',
 5: 'Tuesday',
 6: 'Wednesday'}

In [ ]:
new_df = df.drop(['day_of_week'],axis = 1)

In [ ]:
df.head()

,day_of_week,datehour,lag_no_of_bikes,no_of_bikes,day_of_week_cat
inde_x,,,,,
1 Ave & E 16 St | Friday | 0,Friday,0,0.0,3.0,0
1 Ave & E 16 St | Friday | 2,Friday,2,0.0,1.5,0
1 Ave & E 16 St | Friday | 4,Friday,4,0.0,1.0,0
1 Ave & E 16 St | Friday | 6,Friday,6,0.0,2.5,0
1 Ave & E 16 St | Friday | 7,Friday,7,0.0,5.0,0


**One Hot Encoding**

In [ ]:
#creating dummy values for the weekdays
df_enc = pd.get_dummies(df, columns=["day_of_week_cat"], prefix=["weekday_"] )

In [ ]:
df_enc.head()

,day_of_week,datehour,lag_no_of_bikes,no_of_bikes,weekday__0,weekday__1,weekday__2,weekday__3,weekday__4,weekday__5,weekday__6
inde_x,,,,,,,,,,,
1 Ave & E 16 St | Friday | 0,Friday,0,0.0,3.0,1,0,0,0,0,0,0
1 Ave & E 16 St | Friday | 2,Friday,2,0.0,1.5,1,0,0,0,0,0,0
1 Ave & E 16 St | Friday | 4,Friday,4,0.0,1.0,1,0,0,0,0,0,0
1 Ave & E 16 St | Friday | 6,Friday,6,0.0,2.5,1,0,0,0,0,0,0
1 Ave & E 16 St | Friday | 7,Friday,7,0.0,5.0,1,0,0,0,0,0,0


**Linear Regression**

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [ ]:
X = df_enc[['datehour','lag_no_of_bikes', 'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3', 
             'weekday__4', 'weekday__5', 'weekday__6']].values
y = df_enc['no_of_bikes'].values             

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X,y)

In [ ]:
predictions = lm.predict(X)

In [ ]:
lm.score(X,y)

0.558404332014879

In [ ]:
lm.coef_

array([-0.03504413,  0.68577046,  0.06164362, -0.10333369,  0.13450595,
        0.07767696, -0.09036461, -0.10299637,  0.02286814])

In [ ]:
lm.intercept_

2.013202298134525

By predicting the number of bikes required for the next day on the previous night, we will not be able to use the feature for the “number of bikes picked up from a station in the previous hour” to the full extent. Vital information from the feature will be lost by predicting the bike demand way before the actual time. This is the glaring problem with the approach.


#### **Question 5**

Explain what the author of this code was doing, and why they were doing it.

In [ ]:
  (data.assign(h = lambda x : pd.to_datetime(x.starttime).dt.hour)
  .groupby('h')
  .tripduration
  .agg([np.mean,np.std,len])
  .reset_index()
  .assign(stde = lambda x : x['std'] / np.sqrt(x['len']))).head()

**Answer**

The author is doing an univariate analysis of the tripduration based on the hour of the day. First the hour of the day is calculated from the starttime and then the average, standard deviation, the total trip durations during each hour and grouped by hour.

The author does this to analyze during which hour of the day users tend to spend more time on the trips.